In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
pd.set_option('max_columns', None)

all_players = pd.read_csv('../Data/Basic Player Stats Combined/Basic Player Stats Combined.csv',)
all_players = all_players.drop(labels='Unnamed: 0',axis='columns')
all_players = all_players.sort_values('Year')
all_players.head(2)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,STAR,Year
5067,Marty Byrnes,SF,23.0,LAL,32.0,NaN,6.1,0.8,1.6,0.500,0.0,0.0,NaN,0.8,1.6,0.500,0.500,0.4,0.5,0.867,0.3,0.6,0.8,0.4,0.2,0.0,0.7,1.0,2.0,False,1980
5142,Steve Hawes,PF,29.0,ATL,82.0,NaN,22.6,3.7,7.4,0.502,0.0,0.1,0.375,3.7,7.3,0.504,0.505,1.8,2.2,0.824,1.8,4.2,6.0,1.8,0.9,0.4,1.5,2.5,9.3,False,1980


In [3]:
teams = pd.read_csv('../Data/Team Data/Combined/league_stats.csv',index_col=0)
teams.head(2)

,TEAM,SEASON,AGE,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,PACE,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,ARENA,ATTENDANCE,ATTENDANCE/G
0,CHI,1979-80,25.9,30,52,34,48,-2.71,0.08,-2.63,104.3,107.0,-2.7,101.8,0.373,0.040,0.545,0.489,17.2,32.2,0.291,0.500,15.7,68.0,0.251,Chicago Stadium,NaN,18508.0
1,CHI,1980-81,26.5,45,37,46,36,1.98,0.37,2.34,107.8,105.8,2.0,100.6,0.371,0.026,0.556,0.504,17.2,36.9,0.288,0.493,15.5,68.4,0.232,Chicago Stadium,391118.0,NaN


In [4]:
def fix_years(x):
    return str(x).split('-')[0]
    

In [5]:
# Some formatting to allow the merging
# also removes arena and attendance data
teams.SEASON = teams.SEASON.apply(fix_years)
teams = teams.rename(columns={"SEASON":"Year",'TEAM':'Tm'})
teams = teams.drop(labels=['ARENA','ATTENDANCE', 'ATTENDANCE/G'],axis='columns')

# Adding more descriptors to the team stats (Tm_)
col_list = ['Tm_'+str(i) for i in teams.columns ]
for i in range(len(col_list)):
    if '.1' in col_list[i]:
        col_list[i] = col_list[i].replace('_','_Op_').replace('.1','')

# Fixing columns to be merged on
col_list[0] = 'Tm'
col_list[1] = 'Year'
teams.columns = col_list

    

print(col_list)

['Tm', 'Year', 'Tm_AGE', 'Tm_W', 'Tm_L', 'Tm_PW', 'Tm_PL', 'Tm_MOV', 'Tm_SOS', 'Tm_SRS', 'Tm_ORtg', 'Tm_DRtg', 'Tm_NRtg', 'Tm_PACE', 'Tm_FTr', 'Tm_3PAr', 'Tm_TS%', 'Tm_eFG%', 'Tm_TOV%', 'Tm_ORB%', 'Tm_FT/FGA', 'Tm_Op_eFG%', 'Tm_Op_TOV%', 'Tm_DRB%', 'Tm_Op_FT/FGA']


In [6]:
# Ensures that year columns are strings (Seems to work more simply than DateTime.)

teams.Year = teams.Year.astype(str)
all_players.Year = all_players.Year.astype(str)
teams.head(2)

,Tm,Year,Tm_AGE,Tm_W,Tm_L,Tm_PW,Tm_PL,Tm_MOV,Tm_SOS,Tm_SRS,Tm_ORtg,Tm_DRtg,Tm_NRtg,Tm_PACE,Tm_FTr,Tm_3PAr,Tm_TS%,Tm_eFG%,Tm_TOV%,Tm_ORB%,Tm_FT/FGA,Tm_Op_eFG%,Tm_Op_TOV%,Tm_DRB%,Tm_Op_FT/FGA
0,CHI,1979,25.9,30,52,34,48,-2.71,0.08,-2.63,104.3,107.0,-2.7,101.8,0.373,0.040,0.545,0.489,17.2,32.2,0.291,0.500,15.7,68.0,0.251
1,CHI,1980,26.5,45,37,46,36,1.98,0.37,2.34,107.8,105.8,2.0,100.6,0.371,0.026,0.556,0.504,17.2,36.9,0.288,0.493,15.5,68.4,0.232


In [7]:
# combines the two dataframes. 
# They're sorted by year (1980-2021)
comb = pd.merge(all_players,teams,on=['Tm','Year'],how='left')

# Prints all teams. Should include 'Traded'.
comb.Tm.unique()

array(['LAL', 'ATL', 'WSB', 'PHO', 'BOS', 'HOU', 'IND', 'GSW', 'Traded',
       'DET', 'DEN', 'NYK', 'NJN', 'POR', 'SEA', 'UTA', 'KCK', 'CLE',
       'SDC', 'SAS', 'CHI', 'MIL', 'PHI', 'DAL', 'LAC', 'SAC', 'CHH',
       'MIA', 'ORL', 'MIN', 'VAN', 'TOR', 'WAS', 'MEM', 'NOH', 'CHA',
       'NOK', 'OKC', 'BRK', 'NOP', 'CHO'], dtype=object)

In [8]:
# Printing out the table to verify everything went smoothly.
comb

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,STAR,Year,Tm_AGE,Tm_W,Tm_L,Tm_PW,Tm_PL,Tm_MOV,Tm_SOS,Tm_SRS,Tm_ORtg,Tm_DRtg,Tm_NRtg,Tm_PACE,Tm_FTr,Tm_3PAr,Tm_TS%,Tm_eFG%,Tm_TOV%,Tm_ORB%,Tm_FT/FGA,Tm_Op_eFG%,Tm_Op_TOV%,Tm_DRB%,Tm_Op_FT/FGA
0,Marty Byrnes,SF,23.0,LAL,32.0,NaN,6.10,0.80,1.60,0.500,0.00,0.00,NaN,0.80,1.60,0.500,0.500,0.40,0.50,0.867,0.30,0.60,0.80,0.40,0.20,0.00,0.70,1.00,2.00,False,1980,27.0,54.0,28.0,51.0,31.0,3.84,-0.57,3.27,107.6,103.9,3.7,102.7,0.286,0.013,0.548,0.513,15.8,33.9,0.209,0.468,14.6,64.4,0.208
1,Steve Hawes,PF,29.0,ATL,82.0,NaN,22.60,3.70,7.40,0.502,0.00,0.10,0.375,3.70,7.30,0.504,0.505,1.80,2.20,0.824,1.80,4.20,6.00,1.80,0.90,0.40,1.50,2.50,9.30,False,1980,26.8,31.0,51.0,33.0,49.0,-3.10,0.73,-2.37,104.3,107.3,-3.0,99.4,0.377,0.012,0.537,0.480,16.7,34.1,0.293,0.498,17.3,64.8,0.295
2,Elvin Hayes,PF,34.0,WSB,81.0,NaN,39.30,9.40,20.70,0.454,0.00,0.20,0.231,9.40,20.50,0.456,0.455,4.10,5.90,0.699,3.30,7.70,11.10,1.60,0.80,2.30,2.70,3.80,23.00,True,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spencer Haywood,PF,30.0,LAL,76.0,NaN,20.30,3.80,7.80,0.487,0.00,0.10,0.250,3.80,7.70,0.489,0.488,2.10,2.70,0.772,1.70,2.80,4.60,1.20,0.50,0.80,1.80,2.60,9.70,True,1980,27.0,54.0,28.0,51.0,31.0,3.84,-0.57,3.27,107.6,103.9,3.7,102.7,0.286,0.013,0.548,0.513,15.8,33.9,0.209,0.468,14.6,64.4,0.208
4,Gar Heard,PF,31.0,PHO,82.0,NaN,17.10,2.10,5.00,0.417,0.00,0.00,0.000,2.10,5.00,0.419,0.417,0.80,1.00,0.744,1.40,3.20,4.60,1.20,1.00,0.60,1.10,2.20,5.00,False,1980,25.7,57.0,25.0,55.0,27.0,5.51,-0.68,4.83,104.6,99.4,5.2,104.7,0.332,0.022,0.537,0.492,17.1,35.1,0.247,0.471,17.5,68.2,0.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17648,LeBron James,PG,36.0,LAL,43.0,43.0,33.70,9.30,18.10,0.513,2.30,6.40,0.366,7.00,11.70,0.594,0.578,4.10,5.80,0.701,0.60,7.30,7.90,7.80,1.00,0.60,3.80,1.60,25.00,False,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17649,Justin James,SF,24.0,SAC,31.0,0.0,7.10,1.00,2.10,0.462,0.30,0.80,0.308,0.70,1.30,0.564,0.523,0.40,0.70,0.500,0.10,0.40,0.50,0.50,0.10,0.10,0.30,0.50,2.50,False,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17650,Reggie Jackson,SG,30.0,LAC,62.0,40.0,23.10,3.90,8.80,0.448,1.80,4.00,0.438,2.20,4.70,0.456,0.549,1.00,1.30,0.810,0.40,2.50,2.90,3.20,0.60,0.10,1.10,2.00,10.70,False,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17651,Damian Jones,C,25.0,Traded,66.0,16.0,11.92,1.30,1.87,0.680,0.00,0.09,NaN,1.30,1.80,0.700,0.680,0.99,1.28,0.740,0.78,1.70,2.43,0.51,0.28,0.53,0.51,1.84,3.59,False,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Adding a win% column to account for shorter seasons (strike years, COVID, etc.)

win = comb.Tm_W.to_list()
loss = comb.Tm_L.to_list()

l = list()
for i in range(len(win)):
    if np.isnan(win[i]) or np.isnan(loss[i]):
        l.append(np.nan)
    else:
        l.append(round(win[i]/(win[i] + loss[i]),3))
l[0:10]
comb['Win%'] = l

In [10]:
# Saving as CSV
comb.to_csv('../Data/Combined Player and Team Data/improved_combined_player_and_team_data.csv')